<a href="https://colab.research.google.com/github/lukas1326/webscrap/blob/master/Prices_co_il_scrape_from_colab_to_gsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade -q pygsheets

In [0]:
!pip install --upgrade -q googletrans

In [0]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from googletrans import Translator

In [0]:
import google.auth
from google.colab import auth

auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

In [0]:
gc


# Gredentials to my google file and sheet

In [0]:
# copy key документа google из ссылки https://docs.google.com/spreadsheets/d/1CdYnN62Et7U4q27CnbEfiBg33xYhUC_OxpwsrqbXxOw/edit#gid=1962332589
# между /d/  and /edit
sheet=gc.open_by_key("1CdYnN62Et7U4q27CnbEfiBg33xYhUC_OxpwsrqbXxOw")

In [0]:
# change the sheet number
worksheet = sheet.worksheet_by_title('Sheet6')

# Get soup


In [0]:
#url = "https://www.pricez.co.il/Search.php?q=%D7%97%D7%9C%D7%91+%D7%9E%D7%A8%D7%95%D7%9B%D7%96+%D7%9E%D7%9E%D7%95%D7%AA%D7%A7" - сгущенка
#url = "https://www.pricez.co.il/Alternative-Products.html?APID=1323&Sort=2"# - toffee candy
url = "https://www.pricez.co.il/Category/1/41/69/%D7%92%D7%9C%D7%99%D7%93%D7%94-%D7%A8%D7%92%D7%99%D7%9C%D7%94-%D7%A9%D7%9E%D7%A0%D7%AA" #plain ice cream
html = requests.get(url).text
soup = BeautifulSoup(html)

# Get pictures from site


In [0]:
# the for image without /s/ means the link of bigger picter with good quality
images = []
for i  in soup.select('.ProductImage'):
    for j in i.findAll('img'):
        buz = j['src']
        if "ProductPictures" in buz:
            images.append(buz.replace('/s/','/'))


In [0]:
# Insert images in column "A"
for i in range(len(images)):
  acell = worksheet.cell(f'A{i+1}')
  acell.formula=f'IMAGE("{images[i]}")'

# Get data from site


In [0]:
# Get grom html text of price information
PriceRange = []
for i in soup.select('.ProductPriceRange'):       
    PriceRange.append(i.text)
    


In [0]:
# Get links of all products on page. It needs to get inside all product taking full information (category, product groups)
urlik = []
for i in soup.select('.ProductName'):
    k = i.find_all(href = True)[0]
    urlik.append(f"https://www.pricez.co.il{k['href']}")

In [0]:
# Beautiful soup inside Beautifulsoup
name = []
supplier = []
category = []
product_groups = []
url1 = urlik 
for i in url1:
    soupik = BeautifulSoup(requests.get(i).text)
    be = soupik.select_one('.ProductInfo')
    name.append(be.find('h1').text)
    puk = [ ]
    for j in be.findAll('a'):
        puk.append(j.text)
    supplier.append(puk[0])
    category.append(puk[1])
    product_groups.append(puk[2])

In [0]:
# check the lenths of all fiels 
len(images),len(name),len(PriceRange),len(supplier),len(category),len(product_groups)

In [0]:
# word-to-word translation by google API 
# not optimal, takes much time
# can be overloaded and stopped - should be restart next day
def trans(inList):
    z =[]
    for i in inList:
        k=str(TextBlob(i).translate(to='en'))
        time.sleep(2)
        z.append(k)
    return z

In [0]:
# field translation
NameEng=trans(name)

SupplierNameEng = trans(supplier)
categoryEng  = trans(category)
PriceRangeEng  = trans(PriceRange)
product_groupsEng  = trans(product_groups)


In [0]:
# name in cell ("B")
for i in range(len(NameEng)):
  acell = worksheet.cell(f'B{i+1}')
  acell.set_value(NameEng[i])


In [0]:
# supplier in cell ('C')
for i in range(len(SupplierNameEng)):
  acell = worksheet.cell(f'C{i+1}')
  acell.set_value(SupplierNameEng[i])

In [0]:
# category in cell ("D")
for i in range(len(categoryEng)):
  acell = worksheet.cell(f'D{i+1}')
  acell.set_value(categoryEng[i])

In [0]:
# product group in cell "E"
for i in range(len(product_groupsEng)):
  acell = worksheet.cell(f'E{i+1}')
  acell.set_value(product_groupsEng[i])

In [0]:
# price range in cell "F"
for i in range(len(PriceRangeEng)):
  acell = worksheet.cell(f'F{i+1}')
  acell.set_value(PriceRangeEng[i])

Extracting info from cells

In [0]:
NameEng = worksheet.get_col(2,include_tailing_empty=False)
NameEng

In [0]:
import re

ves = []
for i in NameEng:
  try:
    pattern = re.compile(r'\d{2,3}')
    ves.append(pattern.findall(i)[0])
  except:
    ves.append('')
ves

In [0]:
zhir = []
for i in NameEng:
  pattern = re.compile(r'\s(\S+)\%')
  zhir.append(pattern.findall(i))

zhir

In [0]:
PriceRangeEng = worksheet.get_col(6,include_tailing_empty=False)
for i in range(len(PriceRangeEng)):
  acell = worksheet.cell(f'H{i+1}')
  acell.set_value(PriceRangeEng[i].split()[-5])


In [0]:
for i in range(len(ves)):
  acell = worksheet.cell(f'G{i+1}')
  acell.set_value(ves[i])
